In [ ]:
# Dependencies
import pandas as pd                     # for handling and structuring data
import matplotlib.pyplot as plt         # for creating plots
import scipy.stats as st                # for statistical functions
import numpy as np                      # for generating random numbers and handling numerical operations
from scipy.stats import linregress      # for calculating linear regression

# need to switch to csv
# Load the Excel file
file_path = 'Most Streamed Spotify Songs 2024 - Research.xlsx'
xls = pd.ExcelFile(file_path)

# Display sheet names to identify relevant data
print(xls.sheet_names)


In [ ]:
# Load the 'Likes' sheet
likes_df = pd.read_excel(xls, sheet_name='Likes', skiprows=2)

# Rename 'Row Labels' column
likes_df.rename(columns={'Row Labels': 'Song Title'}, inplace=True)

# Load the 'Views' sheet
views_df = pd.read_excel(xls, sheet_name='Views', skiprows=2)
views_df.rename(columns={'Row Labels': 'Song Title'}, inplace=True)

most_streamed_df = pd.read_excel(xls, sheet_name='Most Streamed Spotify Songs 202')

views_df.head()

In [ ]:
# Ensure that the key columns have consistent names across DataFrames to facilitate merging
likes_df.rename(columns={'Song Title': 'Song'}, inplace=True)
views_df.rename(columns={'Song Title': 'Song'}, inplace=True)

# Select relevant columns
likes_df = likes_df[['Song', 'Sum of TikTok Likes', 'Sum of YouTube Likes']]
views_df = views_df[['Song', 'Max of Spotify Popularity', 'Sum of Spotify Streams', 'Sum of YouTube Views', 'Sum of TikTok Views']]

views_df.head()

In [ ]:
# Combine the two DataFrames based on the 'Song' column
merged_df = pd.merge(likes_df, views_df, on='Song', how='inner')

merged_df.head()

In [ ]:
# Remove rows with missing values
merged_df.dropna(inplace=True)

# Formatting numbers for easy reading 
def format_number(num):
    if abs(num) >= 1_000_000_000:
        return f"{num/1_000_000_000:.1f}B"
    elif abs(num) >= 1_000_000:
        return f"{num/1_000_000:.1f}M"
    elif abs(num) >= 1_000:
        return f"{num/1_000:.1f}K"
    else:
        return f"{num:.1f}"

# Define the columns you want to format
columns_to_format = ['Sum of TikTok Likes', 'Sum of YouTube Likes', 'Sum of Spotify Streams', 'Sum of YouTube Views', 'Sum of TikTok Views']
formatted_df = merged_df

# Apply the formatting to columns
for col in columns_to_format:
    formatted_df[col] = formatted_df[col].apply(format_number)

# Display the updated DataFrame
formatted_df

In [ ]:
# this cell runs foreverrrrr (¬_¬")

# Spotify vs TikTok Views
# Does high TikTok views correlate to high spotify streams?

# Create a scatterplot
plt.scatter(merged_df['Sum of TikTok Views'], merged_df['Sum of Spotify Streams'])
plt.xlabel('Sum of TikTok Views')
plt.ylabel('Sum of Spotify Streams')
plt.title('Spotify vs TikTok Views')
# What kind of editting can be done to make the tick labels not smush together??
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.show()


In [ ]:
# Does the top 500 most viewed TikTok songs have high spotify streaming?

# Create a bar chat

# Sort DataFrame by TikTok Views in descending order
top_500_tiktok_df = merged_df.sort_values(by='Sum of TikTok Views', ascending=False).head(501)

# Remove the 'Grand Total' row
top_500_tiktok_df = top_500_tiktok_df[top_500_tiktok_df['Song'] != 'Grand Total']

# Display top 500 TikTok-viewed songs with their Spotify Streams
top_500_tiktok_df[['Song', 'Sum of TikTok Views', 'Sum of Spotify Streams']]


In [ ]:
# Spotify vs TikTok Views
# Does high TikTok views correlate to high spotify streams?

# Create a scatterplot
plt.scatter(top_500_tiktok_df['Sum of TikTok Views'], top_500_tiktok_df['Sum of Spotify Streams'])
plt.xlabel('Sum of TikTok Views')
plt.ylabel('Sum of Spotify Streams')
plt.title('Spotify vs TikTok Views')
plt.xticks(rotation=90)
plt.yticks(rotation=90)
plt.show()

In [ ]:
# Add the linear regression equation and line to plot
x_values = top_500_tiktok_df["Sum of TikTok Views"]
y_values = top_500_tiktok_df["Sum of Spotify Streams"]

#the output will be a tuple of multiple values  for linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))   # print the equation in my graph (string)

#the scatterplot
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(5.8,0.8),fontsize=15,color="red")

plt.xlabel("Sum of TikTok Views")
plt.ylabel("Sum of Spotify Streams")
plt.title("Spotify vs TikTok Views")

plt.show()

print(f"The r^2-value is: {rvalue**2}")

In [ ]:
# Does the top 10 most viewed TikTok songs have high spotify streaming?

# Create a bar chat

# Sort DataFrame by TikTok Views in descending order
top_ten_tiktok_df = top_500_tiktok_df.sort_values(by='Sum of TikTok Views', ascending=False).head(10)

# Display top 10 TikTok-viewed songs with their Spotify Streams
top_ten_tiktok_df[['Song', 'Sum of TikTok Views', 'Sum of Spotify Streams']]


In [ ]:
# Create a bar chart comparing percentages of TikTok Views and Spotify streaming

# Calculate the percentages
total_tiktok = top_ten_tiktok_df['Sum of TikTok Views'].sum()     # find the total sum first
total_spotify  = top_ten_tiktok_df['Sum of TikTok Views'].sum()

# Convert each song into a percentage of the total
top_ten_tiktok_df['TikTok Percentage'] = (top_ten_tiktok_df['Sum of TikTok Views'] / total_tiktok) * 100
top_ten_tiktok_df['Spotify Percentage'] = (top_ten_tiktok_df['Sum of Spotify Streams'] / total_spotify) * 100

top_ten_tiktok_df

In [ ]:
# Prepare the bar chart
x = np.arange(len(top_ten_tiktok_df['Song']))  # the label locations
width = 0.25                                   # the width of the bars

# Create the subplot
fig, ax = plt.subplots(figsize=(8, 5), layout='constrained')   # constrained to prevent overlap

# Plot the bars
tiktok_bar = ax.bar(x-width/2, top_ten_tiktok_df['Sum of TikTok Views'], width, label='Sum of TikTok Views', color='blue')
spotify_bar = ax.bar(x+width/2, top_ten_tiktok_df['Sum of Spotify Streams'], width, label='Sum of Spotify Streams', color='green')

# Formatting
ax.set_ylabel('Count')
ax.set_title('TikTok Views vs. Spotify Streams')
ax.set_xticks(x)
ax.set_xticklabels(top_ten_tiktok_df['Song'], rotation=30, ha='right')  # Rotate labels for readability
ax.legend()

max_value = max(top_ten_tiktok_df['Sum of Spotify Streams'].max(), top_ten_tiktok_df['Sum of TikTok Views'].max())
ax.set_ylim(0, max_value * 1.1)          # Set y-limit slightly above max value
